In [ ]:
def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)
from csv import DictReader

pointcalls = []
with open('../../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if row['date_fixed'].split('-')[0] == '1787' \
            and row['source_suite'] == "G5" \
            and row['pointcall_function'] == "O":
           # and row['toponyme_fr'] == 'Marseille' \
            pointcalls.append(row)


In [ ]:
from collections import defaultdict
tonnage = defaultdict(float)

for row in pointcalls:
    if row["tonnage"]:
        tonnage[row["toponyme_fr"]] += float(row["tonnage"])
    

In [ ]:
tonnage

In [ ]:
tonnage = sorted([{"port": k, "tonnage": v} for k, v in tonnage.items()], key=lambda x: x["tonnage"], reverse=True)[:20]

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Total des tonnages exportés en 1787",
    "mark": "bar",
    "data": {
        "values": tonnage
    },
    "encoding": {
        "x": {
            "field": "tonnage",
            "type": "quantitative", 
            "title": "tonnage"
        },
        "y": {
            "field": "port",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        }
    }
})

In [ ]:
pointcalls = []
with open('../../data/navigo_all_pointcalls.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for enum, row in enumerate(reader):
        if enum < 5:
            print(row['source_suite'])
        if row['date_fixed'].split('-')[0] == '1787' \
            and (row["source_suite"] == "Registre du petit cabotage (1786-1787)" or \
                 row["source_suite"] == "la Santé registre de patentes de Marseille") \
            and row['pointcall_function'] == "O" \
            and row['net_route_marker'] != "Q":
            pointcalls.append(row)



In [ ]:
source_suites = defaultdict(int)
for row in pointcalls:
    if row["source_suite"]:
        source_suites[row["source_suite"]] += 1

In [ ]:
source_suites

In [ ]:
import pandas as pd

TONNAGE_SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTYdeIwpzaVpY_KS91cXiHxb309iYBS4JN_1_hW-_oyeysuwcIpC2VJ5fWeZJl4tA/pub?output=csv'
estimation_tonnage = pd.read_csv(TONNAGE_SPREADSHEET_URL)

In [ ]:
entrees = pd.DataFrame(pointcalls)
entrees["ship_class_standardized"]

In [ ]:
estimation_tonnage = estimation_tonnage[estimation_tonnage.tonnage_estime_en_tx != "No data"]
estimation_tonnage

In [ ]:
jointure = entrees.merge(estimation_tonnage, 
                           left_on="ship_class_standardized",
                           right_on="ship_class",
                           how="left")

In [ ]:
jointure.tonnage_estime_en_tx.value_counts()

In [ ]:
jointure[jointure.tonnage_estime_en_tx.isna()].ship_class_standardized

In [ ]:
mean = jointure[jointure.tonnage_estime_en_tx.notna()].tonnage_estime_en_tx.astype(int).mean()

In [ ]:
mean

In [ ]:
jointure["tonnage_corrige"] = jointure.tonnage_estime_en_tx.fillna(mean).astype(int)

In [ ]:
jointure["tonnage_corrige"].dtype

In [ ]:
tonnage_total_Marseille = jointure.groupby("toponyme_fr")["tonnage_corrige"].sum()[0]

In [ ]:
tonnage_total_Marseille

In [ ]:
for row in tonnage:
    row["color"] = "sorties"

In [ ]:
tonnage.append({"port": "Marseille", "tonnage": tonnage_total_Marseille, "color": "entrées - tonnage estimé"})

In [ ]:
tonnage

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Comparaison des tonnages en 1787",
    "mark": "bar",
    "data": {
        "values": tonnage
    },
    "encoding": {
        "x": {
            "field": "tonnage",
            "type": "quantitative", 
            "title": "tonnage"
        },
        "y": {
            "field": "port",
            "type": "nominal",
            "title": "port",
            "sort": "-x"
        },
        "color": {"field": "color"}
    }
})